In [279]:
import numpy as np
import pandas as pd
import pickle
from rpy2.robjects import numpy2ri, r, pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import Environment, Formula
numpy2ri.activate()
pandas2ri.activate()

In [280]:
def my_load(fname):
    myfile = open(fname, 'rb')
    x = pickle.load(myfile)
    myfile.close
    return x
X = my_load('X.pkl')
y = my_load('y.pkl')
cols = my_load('cols.pkl')
spots = np.array(my_load('spots.pkl'))
trial_id = np.array(my_load('trial_id.pkl'))
print 'Shapes'
print 'X:', len(X), 'x', X.iloc[0].shape, 'y:', len(y), 'x', y.iloc[0].shape
print 'spots:', spots.shape, 'trial_id:', trial_id.shape

Shapes
X: 1078 x (50, 220) y: 1078 x (50, 1)
spots: (219, 1) trial_id: (1078, 1)


In [281]:
# Convert series to frame
Z = X.to_frame()

In [306]:
# flatten array of 2d arrays
# Nsamp = 5
Nsamp = len(Z)
Nspots = 73
Ncolors = 3
Ntimes = 50
final = np.array(Z['data'].iloc[0])
for i in xrange(1, Nsamp):
    final = np.vstack((final, Z['data'].iloc[i]))
print final.shape

(53900, 220)


In [307]:
# add data, with trial index, to data frame
f = pd.DataFrame(final)

In [308]:
f.iloc[49:52]

,0,1,2,3,4,5,6,7,8,9,...,210,211,212,213,214,215,216,217,218,219
49,980,-35,-95,98,-5,6,0,-17,-19,-22,...,-130,-15,-23,-39,25,47,-47,-19,-17,-25
50,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [309]:
# flatten data
nplong = f.iloc[:,1:].values
print nplong.shape

(53900, 219)


In [310]:
# data column
nplong = nplong.ravel()
print nplong.shape

(11804100,)


In [311]:
print Ncolors*Nspots*Ntimes*Nsamp

11804100


In [312]:
longdf = pd.DataFrame({'DI':nplong})

In [313]:
longdf['color'] = ['R', 'G', 'B']*(Nspots*Ntimes*Nsamp)
print longdf['color'].unique()
longdf.head()

['R' 'G' 'B']


,DI,color
0,0,R
1,0,G
2,0,B
3,0,R
4,0,G


In [314]:
col_nums = list(set([int(x[:-1]) for x in cols[1:]]))
spots = np.array([col_nums[x]*np.ones(Ncolors) for x in range(Nspots)]).flatten()
spots = list(spots)*(Ntimes*Nsamp)
longdf['spot'] = spots
print longdf['spot'].unique()
longdf.head()

[  2.   3.   4.   5.   6.   7.   8.   9.  11.  13.  14.  15.  16.  17.  18.
  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.
  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.  47.  48.  49.
  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  63.  64.  65.
  66.  67.  68.  69.  70.  72.  73.  74.  75.  76.  77.  78.  79.]


,DI,color,spot
0,0,R,2
1,0,G,2
2,0,B,2
3,0,R,3
4,0,G,3


In [315]:
times = np.array([20*x*np.ones(Ncolors*Nspots) for x in range(Ntimes)]).flatten()
times = list(times)*(Nsamp)
longdf['time'] = times
print longdf['time'].unique()
longdf.head()

[   0.   20.   40.   60.   80.  100.  120.  140.  160.  180.  200.  220.
  240.  260.  280.  300.  320.  340.  360.  380.  400.  420.  440.  460.
  480.  500.  520.  540.  560.  580.  600.  620.  640.  660.  680.  700.
  720.  740.  760.  780.  800.  820.  840.  860.  880.  900.  920.  940.
  960.  980.]


,DI,color,spot,time
0,0,R,2,0
1,0,G,2,0
2,0,B,2,0
3,0,R,3,0
4,0,G,3,0


In [316]:
trials = np.array([x*np.ones(Ncolors*Nspots*Ntimes) for x in range(Nsamp)]).flatten()
trials = list(trials)
longdf['trial'] = trials
print longdf['trial'].unique()
longdf.head()

[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   1.07500000e+03
   1.07600000e+03   1.07700000e+03]


,DI,color,spot,time,trial
0,0,R,2,0,0
1,0,G,2,0,0
2,0,B,2,0,0
3,0,R,3,0,0
4,0,G,3,0,0


In [317]:
longdf = pd.get_dummies(longdf, columns=['color'])
longdf.drop('color_B', axis=1, inplace=True)
longdf.head()

,DI,spot,time,trial,color_G,color_R
0,0,2,0,0,0,1
1,0,2,0,0,1,0
2,0,2,0,0,0,0
3,0,3,0,0,0,1
4,0,3,0,0,1,0


In [320]:
tidxs = y.index.values
longdf['trial'] = longdf['trial'].apply(lambda x: tidxs[x])

In [337]:
ylabs = my_load('labels_50.pkl')
# longdf['detection'] = longdf['trial'].apply(lambda x: y.ix[x['trial']]['detection'])

In [338]:
ylabs

array([1, 1, 1, ..., 1, 1, 1])

In [321]:
longdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11804100 entries, 0 to 11804099
Data columns (total 6 columns):
DI         float64
spot       float64
time       float64
trial      int64
color_G    float64
color_R    float64
dtypes: float64(5), int64(1)
memory usage: 630.4 MB


In [326]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()
rX = pandas2ri.py2ri(longdf)
r.assign('longdata', rX)
r("save(longdata, file='longdata.gzip', compress=TRUE)")

rpy2.rinterface.NULL

In [21]:
lme4 = importr('lme4')
base = importr('base')
stats = importr('stats')
fmla = Formula('y ~ x + (1 | id)')
model_env = fmla.environment
model_env['x'] = X
model_env['y'] = y
model_env['id'] = trial_id

In [7]:
model_env['x']

<Matrix - Python:0x10c4a69e0 / R:0x133288000>
[0.000000, 0.000000, 0.000000, ..., -47.000000, -39.000000, -81.000000]

In [24]:
print base.summary(lme4.glmer(fmla, family=binomial, nAGQ=75))

NameError: name 'binomial' is not defined

In [25]:
fmla1 = Formula("y ~ x + (1|id), family=binomial, nAGQ=75")
base.summary(lme4.glmer(fmla1))

RRuntimeError: Error in parse(text = x, keep.source = FALSE) : 
  <text>:1:15: unexpected ','
1: y ~ x + (1|id),
                  ^


In [125]:
cols

Index([u'time', u'2R', u'2G', u'2B', u'3R', u'3G', u'3B', u'4R', u'4G', u'4B',
       ...
       u'76B', u'77R', u'77G', u'77B', u'78R', u'78G', u'78B', u'79R', u'79G',
       u'79B'],
      dtype='object', length=220)

In [126]:
f.columns = cols
f.head()

,time,2R,2G,2B,3R,3G,3B,4R,4G,4B,...,76B,77R,77G,77B,78R,78G,78B,79R,79G,79B
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
f.iloc[:,1:].values.ravel()

(54750,)

In [132]:
np.ravel?

In [135]:
df = pd.DataFrame(columns=['trial', 'time', '1R', '1G', '1B'], data=np.array([[1,0,1,2,3], [1,10,4,5,6],[2,0,7,8,9],[2,10,10,11,12]]))
df.head()

,trial,time,1R,1G,1B
0,1,0,1,2,3
1,1,10,4,5,6
2,2,0,7,8,9
3,2,10,10,11,12


In [136]:
df.iloc[:,2:].values.ravel()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [327]:
nplong2 = final[:,1:]
print nplong2.shape

(53900, 219)
